In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
def outlier_detection(df, ft):
    df.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [3]:
def outliers(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR 
    upper_bound = Q3 + 1.5 * IQR
    filt = (lower_bound>df[ft]) | (upper_bound<df[ft])
    ls = df[filt].index
    return ls

In [4]:
def remove(df, ls):
    ls = list(set(ls))
    df_clean = df.drop(index=ls)
    return df_clean 

In [5]:
df = pd.read_excel('Data.xlsx')
df

,Date,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
0,1990-01-02,200.241,17.24,NaN,NaN,NaN,NaN,0.11,NaN,NaN,NaN,NaN,NaN,1.2146
1,1990-01-03,200.116,18.19,NaN,NaN,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN,1.2096
2,1990-01-04,198.646,19.22,NaN,NaN,NaN,NaN,0.14,NaN,NaN,NaN,NaN,NaN,1.2287
3,1990-01-05,196.850,20.11,NaN,NaN,NaN,NaN,0.20,NaN,NaN,NaN,NaN,NaN,1.2358
4,1990-01-08,197.525,20.26,NaN,NaN,NaN,NaN,0.23,NaN,NaN,NaN,NaN,NaN,1.2452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8366,2022-01-26,2523.635,31.96,251.27,1037.80,344.57,60.75,1.66,87.35,451.50,1829.7,23.5305,103.37,1.1240
8367,2022-01-27,2505.657,30.49,249.65,1079.98,341.10,60.32,1.61,86.61,442.35,1793.1,22.7680,101.66,1.1145
8368,2022-01-28,2566.786,27.66,255.70,1079.16,351.80,60.61,1.59,86.82,431.00,1784.9,22.4727,104.95,1.1151
8369,2022-01-31,2621.974,24.83,261.18,1101.20,363.05,61.77,1.57,88.15,432.45,1795.0,22.4660,106.24,1.1235


In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(df['Date'], inplace=True)
df

,Date,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,,
1990-01-02,1990-01-02,200.241,17.24,NaN,NaN,NaN,NaN,0.11,NaN,NaN,NaN,NaN,NaN,1.2146
1990-01-03,1990-01-03,200.116,18.19,NaN,NaN,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN,1.2096
1990-01-04,1990-01-04,198.646,19.22,NaN,NaN,NaN,NaN,0.14,NaN,NaN,NaN,NaN,NaN,1.2287
1990-01-05,1990-01-05,196.850,20.11,NaN,NaN,NaN,NaN,0.20,NaN,NaN,NaN,NaN,NaN,1.2358
1990-01-08,1990-01-08,197.525,20.26,NaN,NaN,NaN,NaN,0.23,NaN,NaN,NaN,NaN,NaN,1.2452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-26,2022-01-26,2523.635,31.96,251.27,1037.80,344.57,60.75,1.66,87.35,451.50,1829.7,23.5305,103.37,1.1240
2022-01-27,2022-01-27,2505.657,30.49,249.65,1079.98,341.10,60.32,1.61,86.61,442.35,1793.1,22.7680,101.66,1.1145
2022-01-28,2022-01-28,2566.786,27.66,255.70,1079.16,351.80,60.61,1.59,86.82,431.00,1784.9,22.4727,104.95,1.1151


In [7]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
df.dropna(inplace=True)
df = df.iloc[:,1:].apply(lambda x: x.pct_change())

In [9]:
df.dropna(inplace=True)
df

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416
2015-04-01,-0.003747,-0.011772,-0.002422,0.014660,-0.005208,0.007944,-0.036649,0.052311,0.003102,0.021131,0.017678,-0.001423,0.002982
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871
2015-04-06,0.006431,0.004772,0.006456,0.021641,0.008181,0.011326,-0.005263,0.061050,-0.006218,0.014739,0.013026,0.010374,0.003860
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-25,-0.013471,0.042140,-0.013100,0.006176,-0.023181,-0.003748,0.019231,0.027488,0.008499,0.005864,-0.007350,-0.003984,-0.002207
2022-01-26,-0.003061,0.025674,-0.004359,0.015997,-0.001565,-0.006379,0.044025,0.020444,0.014607,-0.012308,-0.011760,-0.015618,-0.005398
2022-01-27,-0.007124,-0.045995,-0.006447,0.040644,-0.010071,-0.007078,-0.030120,-0.008472,-0.020266,-0.020003,-0.032405,-0.016543,-0.008452


#### remove outliers

In [10]:
outlier_list = []
for col in df.columns:
    outlier_list.extend(outliers(df, col))

In [11]:
df_clean = remove(df, outlier_list)
df_clean

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,XAG,VNQ,EURtoUSD
Date,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,-0.004184,-0.008817,-0.009416
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,-0.010587,0.007602,0.010871
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,-0.007974,-0.017618,-0.009888
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,-0.020095,-0.020752,-0.011316
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,0.016703,-0.000727,-0.005160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,-0.001461,0.034115,-0.003645,-0.001406,0.000658,-0.007399,-0.006024,-0.008492,-0.013152,0.000779,0.004489,-0.005764,-0.002993
2022-01-11,0.009908,-0.051031,0.011454,0.019105,0.015022,0.014274,-0.006061,0.038221,0.017693,0.010952,0.013957,0.001540,0.003620
2022-01-14,0.000467,-0.055145,0.000407,-0.014383,0.006223,-0.000935,0.044304,0.020701,-0.027607,-0.002690,-0.005457,-0.009052,-0.003841


In [12]:
pca_data = df_clean.copy()
pca_data_standard = scale(pca_data)

#### PCA

In [13]:
len(df.columns)

13

In [14]:
pca_data_standard

array([[-1.58462898,  1.0958906 , -1.61708224, ..., -0.44869887,
        -1.17368967, -2.30720391],
       [ 0.37452109, -0.388323  ,  0.26929616, ..., -1.04711353,
         0.7961064 ,  2.63239697],
       [-0.6905641 ,  0.18177939, -0.72451775, ..., -0.80285981,
        -2.22944915, -2.42228563],
       ...,
       [-0.16014154, -0.85441   , -0.16798595, ..., -0.56765933,
        -1.20187358, -0.94982702],
       [-2.2217422 ,  1.43974022, -2.21356317, ...,  1.22848662,
        -1.65395193, -0.67999694],
       [-0.7582303 ,  0.59297166, -0.97329447, ..., -1.15664694,
        -1.98951194, -1.3288599 ]])

In [15]:
pca_8 = PCA(n_components=8, random_state=2021)
pca_8.fit(pca_data_standard)

PCA(n_components=8, random_state=2021)

In [16]:
X_pca = pd.DataFrame()
for i in range(1,9):
    pca_data[f'pca_{i}'] = pca_8.transform(pca_data_standard)[:,i-1]

In [17]:
pca_data

,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,...,VNQ,EURtoUSD,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8
Date,,,,,,,,,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,...,-0.008817,-0.009416,4.121680,1.102603,0.177990,0.827063,-0.872144,-0.246058,-1.392309,-0.965144
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,...,0.007602,0.010871,-1.224405,-0.030122,-0.543874,-1.399072,3.612618,-0.653996,-0.884237,-0.073333
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,...,-0.017618,-0.009888,1.279638,0.958936,-2.742603,1.956789,-1.795011,-1.008391,1.031832,-0.078163
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,...,-0.020752,-0.011316,-0.081535,3.101110,-2.270087,1.074117,-1.084319,-0.954664,0.190015,-1.959891
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,...,-0.000727,-0.005160,-1.079130,-0.706271,0.268391,0.701958,-2.061725,0.935439,-0.220683,-0.574905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-10,-0.001461,0.034115,-0.003645,-0.001406,0.000658,-0.007399,-0.006024,-0.008492,-0.013152,0.000779,...,-0.005764,-0.002993,1.972677,0.256181,0.515120,0.313683,-0.422650,0.331879,-0.397025,-1.080102
2022-01-11,0.009908,-0.051031,0.011454,0.019105,0.015022,0.014274,-0.006061,0.038221,0.017693,0.010952,...,0.001540,0.003620,-4.356572,-2.300324,-1.027142,0.850549,-0.184425,-0.089687,0.545777,-0.474724
2022-01-14,0.000467,-0.055145,0.000407,-0.014383,0.006223,-0.000935,0.044304,0.020701,-0.027607,-0.002690,...,-0.009052,-0.003841,0.627691,2.282615,-0.316759,0.175369,-0.372125,2.564142,0.605040,-1.631811


#### clustering 

In [18]:
df_train = pca_data[pca_data.index<pd.to_datetime('20210101')]
df_test = pca_data[pca_data.index>=pd.to_datetime('20210101')]

In [19]:
X_train = df_train.loc[:, 'pca_1':'pca_8']
X_test = df_test.loc[:, 'pca_1':'pca_8']

In [20]:
km = KMeans(n_clusters=2, random_state=2021)
results = km.fit_predict(X_train)
#calculate the Silhouetter Score
score = metrics.silhouette_score(X_train, km.labels_, metric='euclidean')
print(f'Silhouette Score: {score}')

Silhouette Score: 0.19899201010508755


In [21]:
df_train['label'] = results

<ipython-input-21-d4b9ef943818>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['label'] = results


In [22]:
df_train['y'] = df_train['label'].shift(-1)
df_train

<ipython-input-22-ee1dc9e83e3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['y'] = df_train['label'].shift(-1)


,RAY,VIX,IWV,EEM,QQQ,VXUS,T10Y3M,CL1,HG1,GC1,...,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,label,y
Date,,,,,,,,,,,,,,,,,,,,,
2015-03-31,-0.007935,0.053756,-0.008168,-0.001380,-0.010587,-0.010417,-0.005208,-0.022186,-0.014920,-0.001435,...,4.121680,1.102603,0.177990,0.827063,-0.872144,-0.246058,-1.392309,-0.965144,1,0.0
2015-04-02,0.003620,-0.029120,0.002995,0.021534,0.000666,0.009064,0.032609,-0.018966,-0.005276,-0.005960,...,-1.224405,-0.030122,-0.543874,-1.399072,3.612618,-0.653996,-0.884237,-0.073333,0,1.0
2015-04-07,-0.002662,0.002714,-0.002886,0.002809,-0.001698,-0.000579,-0.010582,0.035290,0.016930,-0.006565,...,1.279638,0.958936,-2.742603,1.956789,-1.795011,-1.008391,1.031832,-0.078163,1,1.0
2015-04-09,0.003420,-0.063662,0.003123,0.006617,0.006283,0.003836,0.026455,0.007338,-0.000915,-0.007896,...,-0.081535,3.101110,-2.270087,1.074117,-1.084319,-0.954664,0.190015,-1.959891,1,0.0
2015-04-10,0.004875,-0.038961,0.005190,-0.003161,0.004100,0.001146,0.000000,0.016736,0.001832,0.009216,...,-1.079130,-0.706271,0.268391,0.701958,-2.061725,0.935439,-0.220683,-0.574905,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.002690,-0.076362,0.003165,0.007414,0.004413,0.000506,-0.022989,0.002286,0.002252,0.002774,...,-0.800386,-0.748676,0.013651,0.402261,-0.587029,-0.456220,0.363988,-1.168616,0,0.0
2020-12-28,0.005483,0.007896,0.004958,0.015399,0.010079,0.006065,-0.023529,-0.012648,0.002809,-0.001436,...,-1.671822,-1.000303,0.684349,-0.051951,0.701951,-1.198205,-0.666808,-0.496767,0,1.0
2020-12-29,-0.003945,0.063594,-0.003767,0.006709,0.000895,0.008372,0.012048,0.007980,-0.004762,0.001332,...,0.983672,-0.832157,-1.115059,0.007438,1.114428,0.146621,0.129963,-0.993065,1,0.0


In [23]:
df_train.dropna(inplace=True)

<ipython-input-23-e1854869dff0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.dropna(inplace=True)


In [24]:
df_datarobot = df_train.loc[:, 'pca_1':'pca_8']
df_datarobot['y'] = df_train['y']

In [25]:
df_datarobot

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,y
Date,,,,,,,,,
2015-03-31,4.121680,1.102603,0.177990,0.827063,-0.872144,-0.246058,-1.392309,-0.965144,0.0
2015-04-02,-1.224405,-0.030122,-0.543874,-1.399072,3.612618,-0.653996,-0.884237,-0.073333,1.0
2015-04-07,1.279638,0.958936,-2.742603,1.956789,-1.795011,-1.008391,1.031832,-0.078163,1.0
2015-04-09,-0.081535,3.101110,-2.270087,1.074117,-1.084319,-0.954664,0.190015,-1.959891,0.0
2015-04-10,-1.079130,-0.706271,0.268391,0.701958,-2.061725,0.935439,-0.220683,-0.574905,1.0
...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.954822,-1.811348,-2.047998,-0.050909,-0.119515,0.512682,-0.333356,-0.430446,0.0
2020-12-24,-0.800386,-0.748676,0.013651,0.402261,-0.587029,-0.456220,0.363988,-1.168616,0.0
2020-12-28,-1.671822,-1.000303,0.684349,-0.051951,0.701951,-1.198205,-0.666808,-0.496767,1.0


In [26]:
df_datarobot.to_csv('df_datarobot.csv')

In [27]:
df_datarobot_test = df_test.loc[:, 'pca_1':'pca_8']
df_datarobot_test.to_csv('df_datarobot_test.csv')